# UPI Digital Payments Stress & Revenue Risk Model

## Objective
To quantify the impact of sustained UPI transaction contraction 
and estimate bank-level revenue exposure under structured stress scenarios.

Dataset: NPCI Monthly UPI Statistics  
Period: Mid-2022 to Jan 2026 (latest completed month)

## SECTION 1 — DATA PREPARATION

In [ ]:
import pandas as pd
import numpy as np

# Load cleaned dataset

df = pd.read_csv("upi_cleaned.csv")

# Convert date and sort
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

df.head()


,date,volume_bn,value_Trillion,MoM_vol%,MoM_val%,avg_ticket_size,MA_3M_vol,MA_3M_val
0,2022-06-01,5.8628,10.144126,NaN,NaN,1730.252780,NaN,NaN
1,2022-07-01,6.2884,10.631170,7.259330,4.801242,1690.600153,NaN,NaN
2,2022-08-01,6.5796,10.731622,4.630749,0.944882,1631.044744,6.2436,10.502306
3,2022-09-01,6.7808,11.164381,3.057937,4.032559,1646.469591,6.5496,10.842391
4,2022-10-01,7.3054,12.115875,7.736550,8.522586,1658.482082,6.8886,11.337293


In [2]:
# Month-over-Month Volume Growth
df["MoM_vol%"] = df["volume_bn"].pct_change() * 100

# 3-Month Moving Average
df["MA_3M"] = df["volume_bn"].rolling(3).mean()

# Baseline Statistics
mean_growth = df["MoM_vol%"].mean()
volatility = df["MoM_vol%"].std()

round(mean_growth, 2), round(volatility, 2)

(np.float64(3.21), np.float64(4.95))

### Baseline Ecosystem Profile

- Mean Monthly Growth: ~3.2%
- Historical Volatility (σ): ~4.95%
- Negative Month Frequency: ~30%
- Volume CAGR: ~44%

Interpretation:
UPI exhibits strong structural expansion with moderate cyclical volatility.

## SECTION 2 — STRESS SCENARIO MODELING

In [3]:
latest_volume = df.iloc[-1]["volume_bn"]

# Baseline projection for 3 months
baseline_3M = latest_volume * (1 + mean_growth/100)**3

round(baseline_3M, 2)

np.float64(23.86)

In [4]:
# Stress assumptions
mild_vol = latest_volume * (0.97)**2
moderate_vol = latest_volume * (0.95)**3
severe_vol = latest_volume * (0.93)**6

round(mild_vol, 2), round(moderate_vol, 2), round(severe_vol, 2)

(np.float64(20.42), np.float64(18.61), np.float64(14.04))

In [5]:
mild_gap = baseline_3M - mild_vol
moderate_gap = baseline_3M - moderate_vol
severe_gap = baseline_3M - severe_vol

round(mild_gap, 2), round(moderate_gap, 2), round(severe_gap, 2)

(np.float64(3.44), np.float64(5.25), np.float64(9.82))

In [6]:
mild_relative = round((mild_gap / baseline_3M) * 100, 2)
moderate_relative = round((moderate_gap / baseline_3M) * 100, 2)
severe_relative = round((severe_gap / baseline_3M) * 100, 2)

mild_relative, moderate_relative, severe_relative

(np.float64(14.41), np.float64(22.01), np.float64(41.15))

## SECTION 3 — VALUE IMPACT

In [7]:
latest_ticket = df.iloc[-1]["avg_ticket_size"]

mild_value_gap_bn = mild_gap * latest_ticket
moderate_value_gap_bn = moderate_gap * latest_ticket
severe_value_gap_bn = severe_gap * latest_ticket

# Convert Billion → Trillion
mild_value_gap_T = round(mild_value_gap_bn / 1000, 2)
moderate_value_gap_T = round(moderate_value_gap_bn / 1000, 2)
severe_value_gap_T = round(severe_value_gap_bn / 1000, 2)

mild_value_gap_T, moderate_value_gap_T, severe_value_gap_T

(np.float64(4.49), np.float64(6.85), np.float64(12.82))

## SECTION 4 — REVENUE SENSITIVITY (BANK LEVEL)

### Revenue Sensitivity Assumptions

- Bank Market Share: 12%
- Base Case Fee Realization: 0.10%
- Unit Conversion: 1 Trillion = 100,000 Crore

In [8]:
market_share = 0.12
fee_rate = 0.001  # 0.10%

# Convert Trillion → Crore
moderate_gap_crore = moderate_value_gap_T * 100000

bank_revenue_risk = moderate_gap_crore * fee_rate * market_share

round(bank_revenue_risk, 2)

np.float64(82.2)

## SECTION 5 — STATISTICAL CONTEXT

In [9]:
z_mild = round((-3 - mean_growth) / volatility, 2)
z_moderate = round((-5 - mean_growth) / volatility, 2)
z_severe = round((-7 - mean_growth) / volatility, 2)

z_mild, z_moderate, z_severe

(np.float64(-1.25), np.float64(-1.66), np.float64(-2.06))

### Statistical Interpretation

- Mild: ~ -1.25σ (Unusual)
- Moderate: ~ -1.66σ (Rare)
- Severe: ~ -2.06σ (Tail Event)

Single-month contraction may fall within volatility band, 
but sustained multi-month contraction produces structural deviation.

## SECTION 6 — EARLY WARNING FRAMEWORK

In [10]:
warning_threshold = mean_growth - volatility
critical_threshold = mean_growth - 2 * volatility

round(warning_threshold, 2), round(critical_threshold, 2)

(np.float64(-1.74), np.float64(-6.69))

In [11]:
def classify_stress(x):
    if x <= critical_threshold:
        return "Critical"
    elif x <= warning_threshold:
        return "Watchlist"
    else:
        return "Normal"

df["Stress_Level"] = df["MoM_vol%"].apply(classify_stress)

df["Stress_Level"].value_counts()

Stress_Level
Normal       39
Watchlist     5
Name: count, dtype: int64

## SECTION 7 — FINAL CONSOLIDATED RISK TABLE

In [12]:
final_table = pd.DataFrame({
    "Scenario": ["Mild", "Moderate", "Severe"],
    "Relative Gap (%)": [mild_relative, moderate_relative, severe_relative],
    "Value Impact (₹ Trillion)": [mild_value_gap_T, moderate_value_gap_T, severe_value_gap_T],
    "Z Score": [z_mild, z_moderate, z_severe],
    "Bank Revenue Risk (₹ Cr, 0.1%)": [
        round(mild_value_gap_T * 100000 * fee_rate * market_share, 2),
        round(moderate_value_gap_T * 100000 * fee_rate * market_share, 2),
        round(severe_value_gap_T * 100000 * fee_rate * market_share, 2)
    ]
})

final_table


,Scenario,Relative Gap (%),Value Impact (₹ Trillion),Z Score,"Bank Revenue Risk (₹ Cr, 0.1%)"
0,Mild,14.41,4.49,-1.25,53.88
1,Moderate,22.01,6.85,-1.66,82.20
2,Severe,41.15,12.82,-2.06,153.84


## SECTION 8 — EXECUTIVE SUMMARY

#### Executive Summary

- UPI Mean MoM Growth: ~3.2%
- Historical Volatility: ~4.95%
- Moderate Stress: ~22% deviation from expected trajectory
- Value Impact: ~₹6.85 Trillion
- Bank Exposure (12% share, 0.1% yield): ~₹82 Crore
- Early Warning triggers below -1σ contraction

Conclusion:
Sustained transaction contraction can materially impact revenue exposure, 
and volatility-adjusted monitoring enables proactive risk management.
